In [2]:
import numpy as np
import tensorflow as tf

from keras.layers import Add, Conv2D, Input, Lambda, Activation
from keras.models import Model

/home/chandu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def SubpixelConv2D(scale, **kwargs):
    return Lambda(lambda x: tf.depth_to_space(x, scale), **kwargs)

In [4]:
def edsr(x_in, scale, num_filters=64, num_res_blocks=8, res_block_scaling=None, tanh_activation=False):
    
    x = x_in
    x = b = Conv2D(num_filters, 3, padding='same')(x)
    
    for i in range(num_res_blocks):
        b = res_block(b, num_filters, res_block_scaling)
        
    b = Conv2D(num_filters, 3, padding='same')(b)
    x = Add()([x, b])

    x = upsample(x, scale, num_filters)
    x = Conv2D(1, 3, padding='same')(x)

    if tanh_activation:
        x = Activation('tanh')(x)

    return Model(x_in, x, name="edsr")


def res_block(x_in, filters, scaling):
    x = Conv2D(filters, 3, padding='same')(x_in)
    x = Activation('relu')(x)
    x = Conv2D(filters, 3, padding='same')(x)
    x = Add()([x_in, x])
    if scaling:
        x = Lambda(lambda t: t * scaling)(x)
    return x


def upsample(x, scale, num_filters):
    def upsample_1(x, factor, **kwargs):
        x = Conv2D(num_filters * (factor ** 2), 3, padding='same', **kwargs)(x)
        return SubpixelConv2D(factor)(x)

    if scale == 2:
        x = upsample_1(x, 2, name='conv2d_1_scale_2')
    elif scale == 3:
        x = upsample_1(x, 3, name='conv2d_1_scale_3')
    elif scale == 4:
        x = upsample_1(x, 2, name='conv2d_1_scale_2')
        x = upsample_1(x, 2, name='conv2d_2_scale_2')

    return x

In [5]:
# model = edsr(scale=2)
# model.compile(optimizer='adam', loss='mse')
# model.summary()

TypeError: edsr() missing 1 required positional argument: 'x_in'